In [1]:
RunningInCOLAB = 'google.colab' in str(get_ipython())
if RunningInCOLAB:
    !git clone https://github.com/MJC598/Neuron_Burst_Analysis.git

In [2]:
%matplotlib notebook
import torch.nn as nn
import numpy as np
import torch
torch.manual_seed(0)

from torch.utils.data import TensorDataset, DataLoader
import scipy.io
import random
import pandas as pds
import time

from scipy import stats
from sklearn.metrics import r2_score
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

### Class Explanations

These are 3 regression RNN-based models. In order to change it to a classifier the 
nn.Linear layers must have their second parameter changed to match the number of 
expected outputs.

* Expected Input Shape: (batch_size, time_sequence, features)

* Input_Size - number of features
* Hidden_Size - number of connections between the hidden layers
* Batch_Size - How many samples you want to push through the network before executing backprop
    (this is a hyperparameter that can change how fast or slow a model converges)
* Batch_First - Should always be set to True to keep input shape the same
* Dropout - Only really does anything with more than 1 layer on the LSTM, RNN, GRU. Useful to help generalize training

In [3]:
class baselineRNN(nn.Module):
    def __init__(self,input_size,hidden_size,output_size=1,
                 batch_size=1,num_layers=1,batch_first=True,dropout=0.0):
        super(baselineRNN, self).__init__()
        self.rnn1 = nn.RNN(input_size=input_size,hidden_size=hidden_size,
                           num_layers=num_layers,batch_first=batch_first,dropout=dropout)
        self.lin = nn.Linear(hidden_size,output_size)
        self.h0 = torch.randn(num_layers, batch_size, hidden_size)

    def forward(self, x):
        x, h_n  = self.rnn1(x,self.h0)

        # take last cell output
        out = self.lin(x[:, -1, :])

        return out

class baselineLSTM(nn.Module):
    def __init__(self,input_size,hidden_size,output_size=1,
                 batch_size=1,num_layers=1,batch_first=True,dropout=0.0):
        super(baselineLSTM, self).__init__()
        self.rnn = nn.LSTM(input_size=input_size,hidden_size=hidden_size,
                           num_layers=num_layers,batch_first=batch_first,dropout=dropout)
        self.lin = nn.Linear(hidden_size,output_size)
        self.h0 = torch.randn(num_layers, batch_size, hidden_size)
        self.c0 = torch.randn(num_layers, batch_size, hidden_size)

    def forward(self, x):
        x, (h_n, c_n)  = self.rnn(x,(self.h0,self.c0))

        # take last cell output
        out = self.lin(x[:, -1, :])

        return out

class baselineGRU(nn.Module):
    def __init__(self,input_size,hidden_size,output_size=1,
                 batch_size=1,num_layers=1,batch_first=True,dropout=0.0):
        super(baselineGRU, self).__init__()
        self.rnn = nn.GRU(input_size=input_size,hidden_size=hidden_size,
                          num_layers=num_layers,batch_first=batch_first,dropout=dropout)
        self.lin = nn.Linear(hidden_size,output_size)
        self.h0 = torch.randn(num_layers, batch_size, hidden_size)

    def forward(self, x):
        # print(self.h0.shape)
        x, h_n  = self.rnn(x,self.h0)

        # take last cell output
        out = self.lin(x[:, -1, :])

        return out
    
class conv1DLSTM(nn.Module):
    def __init__(self,input_size,hidden_size,output_size=1,
                 batch_size=1,num_layers=1,batch_first=True,dropout=0.0):
        super(conv1DLSTM, self).__init__()
        self.c1 = nn.Conv1d(input_size, hidden_size, 3)
        self.p1 = nn.AvgPool1d(3)
        self.c2 = nn.Conv1d(hidden_size, hidden_size, 2)
        self.p2 = nn.AvgPool1d(2)
        self.c3 = nn.Conv1d(hidden_size, hidden_size, 1)
#         self.p1 = nn.AvgPool1d(2)
        self.sigmoid = nn.Sigmoid()
        self.tanh = nn.Tanh()
        self.dropout = nn.Dropout(p=0.2)
        self.rnn = nn.LSTM(input_size=hidden_size,hidden_size=hidden_size,
                           num_layers=num_layers,batch_first=batch_first,dropout=dropout)
        self.lin = nn.Linear(hidden_size,output_size)
        self.h0 = torch.randn(num_layers, batch_size, hidden_size)
        self.c0 = torch.randn(num_layers, batch_size, hidden_size)
        
    def forward(self, x):
        
        #switch (batch, sequence, feature) to (batch, feature, sequence)
        x = x.transpose(1,2)
        x = self.dropout(x)
        x = self.c1(x)
        x = self.p1(x)
#         x = self.tanh(x)
#         x = self.c2(x)
#         x = self.p2(x)
#         x = self.dropout(x)
        x = self.c3(x)
        x = self.sigmoid(x)
        
        #switch backwards
        x = x.transpose(1,2)
        x, (h_n, c_n)  = self.rnn(x,(self.h0,self.c0))

        # take last cell output
        out = self.lin(x[:, -1, :])

        return out
    
class conv1DGRU(nn.Module):
    def __init__(self,input_size,hidden_size,output_size=1,
                 batch_size=1,num_layers=1,batch_first=True,dropout=0.0):
        super(conv1DGRU, self).__init__()
        self.c1 = nn.Conv1d(input_size, hidden_size, 5)
        self.p1 = nn.AvgPool1d(5)
        self.c2 = nn.Conv1d(hidden_size, hidden_size, 3)
        self.p2 = nn.AvgPool1d(3)
        self.c3 = nn.Conv1d(hidden_size, hidden_size, 1)
#         self.p1 = nn.AvgPool1d(2)
        self.sigmoid = nn.Sigmoid()
        self.tanh = nn.Tanh()
        self.dropout = nn.Dropout(p=0.2)
        self.rnn = nn.GRU(input_size=hidden_size,hidden_size=hidden_size,
                           num_layers=num_layers,batch_first=batch_first,dropout=dropout)
        self.lin = nn.Linear(hidden_size,output_size)
        self.h0 = torch.randn(num_layers, batch_size, hidden_size)
        
    def forward(self, x):
        
        #switch (batch, sequence, feature) to (batch, feature, sequence)
        x = x.transpose(1,2)
        x = self.dropout(x)
        x = self.c1(x)
        x = self.p1(x)
        x = self.c2(x)
#         x = self.p2(x)
#         x = self.c3(x)
        x = self.sigmoid(x)
        
        #switch backwards
        x = x.transpose(1,2)
        x, h_n  = self.rnn(x,self.h0)

        # take last cell output
        out = self.lin(x[:, -1, :])

        return out

In [4]:
TIMESTEPS = 80
FRONT_TIME = -25
BACK_TIME = 5
T_START = 50+FRONT_TIME
T_END = 50+BACK_TIME
MODEL = conv1DLSTM
OUTPUT = 'full'
LOSS_FILE = ('losses/bursts/losses_' + str(MODEL) + 
             '_' + OUTPUT + '_' + str(FRONT_TIME) + 
             '_' + str(T_END) + '_fullin.csv')
PATH = ('models/bursts/' + str(MODEL) + '_' + OUTPUT + 
        '_' + str(FRONT_TIME) + '_' + str(T_END) + 
        '_fullin.pth')
DATA_PATH = 'data/info_collect_for_NN_network_burst_separatePNITN.mat'
COLAB_PRE = 'Neuron_Burst_Analysis/'
if RunningInCOLAB:
    LOSS_FILE = COLAB_PRE + LOSS_FILE
    PATH = COLAB_PRE + PATH
    DATA_PATH = COLAB_PRE + DATA_PATH

# Specific Model Parameters
input_size = 8
hidden_size = 20
output_size = 1
batch_size = 32
num_layers = 1
batch_first = True
dropout = 0.0
epochs = 35

In [14]:
def get_data_from_mat(file_path, output_index=None, type='pre_pn'):
    data = scipy.io.loadmat(file_path)
    duration = []
    amp = []
    pb_fr_pn = []
    pb_fr_itn = []
    pb_aff_pn = []
    pb_aff_itn = []
    pb_exc_pn = []
    pb_inh_pn = []
    pb_exc_itn = []
    pb_inh_itn = []
    wb_fr_pn = []
    wb_fr_itn = []
    wb_aff_pn = []
    wb_aff_itn = []
    wb_exc_pn = []
    wb_inh_pn = []
    wb_exc_itn = []
    wb_inh_itn = []

    print(data['info_collect'][0])
#     print(data['info_collect'].shape)
    for i in range(1, data['info_collect'].shape[0]):
        arr = data['info_collect'][i]
        
#         print(arr.shape)
        
        duration.append(arr[0])
        amp.append(arr[1])
        
        pb_fr_pn.append(arr[2])
        pb_fr_itn.append(arr[3])
        pb_aff_pn.append(arr[4])
        pb_aff_itn.append(arr[5])
        pb_exc_pn.append(arr[6])
        pb_inh_pn.append(arr[7])
        pb_exc_itn.append(arr[8])
        pb_inh_itn.append(arr[9])
        
#         print(arr[10].shape)
        wb_fr_pn.append(arr[10])
        wb_fr_itn.append(arr[11])
        wb_aff_pn.append(arr[12])
        wb_aff_itn.append(arr[13])
        wb_exc_pn.append(arr[14])
        wb_inh_pn.append(arr[15])
        wb_exc_itn.append(arr[16])
        wb_inh_itn.append(arr[17])
    
    print(np.array(pb_fr_pn).shape)
    print(np.array(pb_fr_itn).shape)
    print(np.array(pb_aff_pn).shape)
    print(np.array(pb_aff_itn).shape)
    print(np.array(pb_exc_pn).shape)
    print(np.array(pb_inh_pn).shape)
    print(np.array(pb_exc_itn).shape)
    print(np.array(pb_inh_itn).shape)
    
    print(np.array(wb_fr_pn).shape)
    print(np.array(wb_fr_itn).shape)
    print(np.array(wb_aff_pn).shape)
    print(np.array(wb_aff_itn).shape)
    print(np.array(wb_exc_pn).shape)
    print(np.array(wb_inh_pn).shape)
    print(np.array(wb_exc_itn).shape)
    print(np.array(wb_inh_itn).shape)
    
    full_labels = np.concatenate((duration, amp), axis=2)
    
    front_data = np.concatenate((pb_fr_pn, pb_fr_itn, pb_aff_pn, pb_aff_itn, 
                                pb_exc_pn, pb_inh_pn, pb_exc_itn, pb_inh_itn), axis=2)
    
    rear_data = np.concatenate((wb_fr_pn, wb_fr_itn, wb_aff_pn, wb_aff_itn,
                                wb_exc_pn, wb_inh_pn, wb_exc_itn, wb_inh_itn), axis=2)
    
    for j in range(2):
        x = full_labels[:,:,j]
        full_labels[:,:,j] = (x - np.min(x))/(np.max(x)-np.min(x))
    
    for i in range(full_data.shape[0]):
        for j in range(input_size):
            x = full_data[i,:,j]
            full_data[i,:,j] = (x - np.min(x))/(np.max(x)-np.min(x))
    
    print(full_data.shape)
    
    random.seed(10)
    data_samples = 24832 #24848
    k = 19808
    full = np.arange(data_samples)
    training_indices = np.random.choice(full, size=k, replace=False)
    validation_indices = np.delete(full,training_indices)
    
    training_data = full_data[training_indices,T_START:T_END,:] 
    validation_data = full_data[validation_indices,T_START:T_END,:]
    
    if output_index is None:
        training_labels = full_labels[training_indices,:,:] 
    else:
        training_labels = full_labels[training_indices,:,output_index]
    
    if output_index is None:
        validation_labels = full_labels[validation_indices,:,:]
    else:
        validation_labels = full_labels[validation_indices,:,output_index]
    
#     print(training_data.shape)
#     print(training_labels.shape)
#     print(validation_data.shape)
#     print(validation_labels.shape)

    training_dataset = TensorDataset(torch.Tensor(training_data), torch.Tensor(training_labels))
    validation_dataset = TensorDataset(torch.Tensor(validation_data), torch.Tensor(validation_labels))

    return training_dataset, validation_dataset
get_data_from_mat(DATA_PATH)

[array(['burst duration(ms)'], dtype='<U18')
 array(['burst amp (uV)'], dtype='<U14')
 array(['pre-burst PN FR in 50ms window'], dtype='<U30')
 array(['pre-burst ITN FR in 50ms window'], dtype='<U31')
 array(['pre-burst AFF Rate to PN in 50ms window'], dtype='<U39')
 array(['pre-burst AFF Rate to ITN in 50ms window'], dtype='<U40')
 array(['pre-burst exc noise conduct (nS) of PN in 50ms window'],
      dtype='<U53')
 array(['pre-burst inh noise conduct (nS) of PN in 50ms window'],
      dtype='<U53')
 array(['pre-burst exc noise conduct (nS) of ITN in 50ms window'],
      dtype='<U54')
 array(['pre-burst inh noise conduct (nS) of ITN in 50ms window'],
      dtype='<U54')
 array(['within-burst PN FR'], dtype='<U18')
 array(['ITN FR within-burst '], dtype='<U20')
 array(['AFF Rate to PN within-burst'], dtype='<U27')
 array(['AFF Rate to ITN within-burst'], dtype='<U28')
 array(['exc noise conduct (nS) of PN within-burst'], dtype='<U41')
 array(['inh noise conduct (nS) of PN within-burst'

<ipython-input-14-1e52ce872a94>:60: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  print(np.array(wb_fr_pn).shape)
<ipython-input-14-1e52ce872a94>:61: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  print(np.array(wb_fr_itn).shape)
<ipython-input-14-1e52ce872a94>:62: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  print(np.array(wb_aff_pn).

AxisError: axis 2 is out of bounds for array of dimension 1

### Training Method
* Model - Model initialized based on classes above
* Save_Filepath - Where you want to save the model to. Should end with a .pt or .pth extension. This is how you are able to load the model later for testing, etc.
* training_loader - dataloader iterable with training dataset samples
* validation_loader - dataloader iterable with validation dataset samples

In [ ]:
def train_model(model,save_filepath,training_loader,validation_loader,epochs,count):
    
    epochs_list = []
    train_loss_list = []
    val_loss_list = []
    training_len = len(training_loader.dataset)
    validation_len = len(validation_loader.dataset)

    #splitting the dataloaders to generalize code
    data_loaders = {"train": training_loader, "val": validation_loader}

    """
    This is your optimizer. It can be changed but Adam is generally used. 
    Learning rate (alpha in gradient descent) is set to 0.001 but again 
    can easily be adjusted if you are getting issues

    Loss function is set to Mean Squared Error. If you switch to a classifier 
    I'd recommend switching the loss function to nn.CrossEntropyLoss(), but this 
    is also something that can be changed if you feel a better loss function would work
    """
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    loss_func = nn.MSELoss()
#     loss_func = nn.L1Loss()
    decay_rate = 0.93 #decay the lr each step to 93% of previous lr
    lr_sch = torch.optim.lr_scheduler.ExponentialLR(optimizer=optimizer, gamma=decay_rate)

    total_start = time.time()

    """
    You can easily adjust the number of epochs trained here by changing the number in the range
    """
    for epoch in tqdm(range(epochs), position=0, leave=True):
        start = time.time()
        train_loss = 0.0
        val_loss = 0.0
        temp_loss = 100000000000000.0
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train(True)
            else:
                model.train(False)

            running_loss = 0.0
            for i, (x, y) in enumerate(data_loaders[phase]):  
                output = model(x)                       
                loss = loss_func(torch.squeeze(output), torch.squeeze(y))  
                #backprop             
                optimizer.zero_grad()           
                if phase == 'train':
                    loss.backward()
                    optimizer.step()                                      

                #calculating total loss
                running_loss += loss.item()
            
            if phase == 'train':
                train_loss = running_loss
                lr_sch.step()
            else:
                val_loss = running_loss

        end = time.time()
        # shows total loss
        if epoch%10 == 0:
            print('[%d, %5d] train loss: %.6f val loss: %.6f' % (epoch + 1, i + 1, train_loss, val_loss))
#         print(end - start)
        
        #saving best model
        if val_loss < temp_loss:
            torch.save(model, save_filepath)
            temp_loss = val_loss
        epochs_list.append(epoch)
        train_loss_list.append(train_loss)
        val_loss_list.append(val_loss)
    total_end = time.time()
#     print(total_end - total_start)
    #Creating loss csv
    loss_df = pds.DataFrame(
        {
            'epoch': epochs_list,
            'training loss': train_loss_list,
            'validation loss': val_loss_list
        }
    )
    # Writing loss csv, change path to whatever you want to name it
    lf = ('losses/losses_' + str(MODEL) + '_' 
          + OUTPUT + '_' + str(FRONT_TIME) + '_' 
          + str(T_END) + str(count) + '_fullin.csv')
    loss_df.to_csv(lf, index=None)
    return train_loss_list, val_loss_list

### R2 Scoring
* Model - same model as sent to train_model
* testing_dataloader - whichever dataloader you want to R2 Score

In [ ]:
def r2_score_eval(model, testing_dataloader):
    output_list = []
    labels_list = []
    for i, (x, y) in enumerate(testing_dataloader):
        output = model(x) 
        output_list.append(np.transpose(output.detach().cpu().numpy()))
        labels_list.append(np.transpose(y.detach().cpu().numpy()))
    output_list = np.transpose(np.hstack(output_list))
    labels_list = np.transpose(np.hstack(labels_list)).reshape((-1,output_size))
#     print(output_list.shape)
#     print(labels_list.shape)
    print(r2_score(np.squeeze(labels_list), output_list))
    return output_list, labels_list

### Program Start

In [ ]:
model1 = MODEL(input_size,hidden_size,output_size,batch_size,num_layers,batch_first,dropout)
model2 = MODEL(input_size,hidden_size,output_size,batch_size,num_layers,batch_first,dropout)
model3 = MODEL(input_size,hidden_size,output_size,batch_size,num_layers,batch_first,dropout)

a1_training_dataset, a1_validation_dataset = get_data_from_mat(DATA_PATH, 0) #retrieve amps1
a2_training_dataset, a2_validation_dataset = get_data_from_mat(DATA_PATH, 1) #retrieve amps2
d_training_dataset, d_validation_dataset = get_data_from_mat(DATA_PATH, 2) #retrieve duration
# b_training_dataset, b_validation_dataset = get_data_from_mat(DATA_PATH, 3) #retrieve burst

# Turn datasets into iterable dataloaders
a1_training_loader = DataLoader(dataset=a1_training_dataset,batch_size=batch_size,shuffle=True)
a1_validation_loader = DataLoader(dataset=a1_validation_dataset,batch_size=batch_size)

a2_training_loader = DataLoader(dataset=a2_training_dataset,batch_size=batch_size,shuffle=True)
a2_validation_loader = DataLoader(dataset=a2_validation_dataset,batch_size=batch_size)

d_training_loader = DataLoader(dataset=d_training_dataset,batch_size=batch_size,shuffle=True)
d_validation_loader = DataLoader(dataset=d_validation_dataset,batch_size=batch_size)


p1 = 'models/' + str(MODEL) + '_' + OUTPUT + '_' + str(FRONT_TIME) + '_' + str(T_END) + str(0) + '_fullin.pth'
a1_training_loss, a1_validation_loss = train_model(model1,p1,a1_training_loader,a1_validation_loader,epochs,0)
p2 = 'models/' + str(MODEL) + '_' + OUTPUT + '_' + str(FRONT_TIME) + '_' + str(T_END) + str(1) + '_fullin.pth'
a2_training_loss, a2_validation_loss = train_model(model2,p2,a2_training_loader,a2_validation_loader,epochs,1)
p3 = 'models/' + str(MODEL) + '_' + OUTPUT + '_' + str(FRONT_TIME) + '_' + str(T_END) + str(2) + '_fullin.pth'
d_training_loss, d_validation_loss = train_model(model3,p3,d_training_loader,d_validation_loader,epochs,2)


model1 = torch.load(p1)
model1.eval()
a1_t_output_list, a1_t_labels_list = r2_score_eval(model1, a1_training_loader)
a1_v_output_list, a1_v_labels_list = r2_score_eval(model1, a1_validation_loader)

model2 = torch.load(p2)
model2.eval()
a2_t_output_list, a2_t_labels_list = r2_score_eval(model2, a2_training_loader)
a2_v_output_list, a2_v_labels_list = r2_score_eval(model2, a2_validation_loader)

model3 = torch.load(p3)
model3.eval()
d_t_output_list, d_t_labels_list = r2_score_eval(model3, d_training_loader)
d_v_output_list, d_v_labels_list = r2_score_eval(model3, d_validation_loader)


In [ ]:
fig, ax = plt.subplots(nrows=4, ncols=2)
fig.tight_layout()
ax[0,0].plot(range(epochs), a1_training_loss)
ax[0,0].set_title('Validation Loss')
ax[0,0].set_ylabel('Loss')
ax[0,0].set_xlabel('Epoch')

ax[0,1].plot(range(epochs), a1_validation_loss)
ax[0,1].set_title('Training Loss')
ax[0,1].set_ylabel('Loss')
ax[0,1].set_xlabel('Epoch')


ax[1,0].plot(np.arange(a1_v_labels_list.shape[0]), a1_v_labels_list[:,0], color='blue')
ax[1,0].plot(np.arange(a1_v_labels_list.shape[0]), a1_v_output_list[:,0], color='red')
ax[1,0].set_title('Validation Amps1 per Sample')
ax[1,0].set_ylabel('Amp1')
ax[1,0].set_xlabel('Sample')

ax[1,1].plot(np.arange(a1_t_labels_list.shape[0]), a1_t_labels_list[:,0], color='blue')
ax[1,1].plot(np.arange(a1_t_labels_list.shape[0]), a1_t_output_list[:,0], color='red')
ax[1,1].set_title('Training Amps1 per Sample')
ax[1,1].set_ylabel('Amp1')
ax[1,1].set_xlabel('Sample')

ax[2,0].plot(np.arange(a2_v_labels_list.shape[0]), a2_v_labels_list[:,0], color='blue')
ax[2,0].plot(np.arange(a2_v_labels_list.shape[0]), a2_v_output_list[:,0], color='red')
ax[2,0].set_title('Validation Amps2 per Sample')
ax[2,0].set_ylabel('Amp2')
ax[2,0].set_xlabel('Sample')

ax[2,1].plot(np.arange(a2_t_labels_list.shape[0]), a2_t_labels_list[:,0], color='blue')
ax[2,1].plot(np.arange(a2_t_labels_list.shape[0]), a2_t_output_list[:,0], color='red')
ax[2,1].set_title('Training Amps2 per Sample')
ax[2,1].set_ylabel('Amp2')
ax[2,1].set_xlabel('Sample')

ax[3,0].plot(np.arange(d_v_labels_list.shape[0]), d_v_labels_list[:,0], color='blue')
ax[3,0].plot(np.arange(d_v_labels_list.shape[0]), d_v_output_list[:,0], color='red')
ax[3,0].set_title('Validation Duration per Sample')
ax[3,0].set_ylabel('Duration')
ax[3,0].set_xlabel('Sample')

ax[3,1].plot(np.arange(d_t_labels_list.shape[0]), d_t_labels_list[:,0], color='blue')
ax[3,1].plot(np.arange(d_t_labels_list.shape[0]), d_t_output_list[:,0], color='red')
ax[3,1].set_title('Training Duration per Sample')
ax[3,1].set_ylabel('Duration')
ax[3,1].set_xlabel('Sample')

plt.show()